<a href="https://colab.research.google.com/github/danieljai/CIND820-AndyLee/blob/main/AndyLee_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Outstanding tasks
- remove retweets RT @
- convert html entities to their own
- remove hyperlinks
- remove \n
- remove tweet ID mentions
- remove non-English tweets
- write code to allow easier future presentation
- study NLP lectures to cover gaps in knwoledge
- GCP Google Cloud - COLAB, should be able to use it under Ryerson
- keep notebook diary 

11/11
- add readme file explaining rehydration process
- also explain anything that was done outside of python

# Preload setup

In [4]:
%config IPCompleter.greedy=True
import pandas as pd
pd.set_option('display.max_colwidth', 200)
pd.options.display.max_colwidth = 500
pd.options.display.max_rows = 100

# Import Dataset

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:


#df = pd.read_csv('Book1-fastsave.csv')
df = pd.read_csv("/content/drive/My Drive/__CIND 820 - Data Analytics Project/3-data/Book1-fastsave.csv")


- Setting display options
- previewing the text attribute

# Data Cleaning and Manipulation

## Readjust attribute datatype

For `retweet_id`, `in_reply_to_status_id`, `in_reply_to_user_id`
- Convert `null` values to 0
- Convert attribute as int64

In [7]:
df.retweet_id = df[df['retweet_id'].notnull()].retweet_id.astype('int64') 

In [8]:
df.retweet_id = df.retweet_id.fillna(0).astype('int64')
df.in_reply_to_status_id = df.in_reply_to_status_id.fillna(0).astype('int64')
df.in_reply_to_user_id = df.in_reply_to_user_id.fillna(0).astype('int64')

## Misc. cleaning up to reduce noise when conducting sentimental analysis
1. remove \n
2. remove URL
3. remove user referrals
4. remove hashtags

In [9]:
df['modified_text'] = df.text.str.replace(r'\n', '')
df['modified_text'] = df.modified_text.str.replace(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '')
df['modified_text'] = df.modified_text.str.replace(r'\B@\w+', '')
df['modified_text'] = df.modified_text.str.replace(r'\B#\w+', '')

## Splitting Dataframes (originals and retweets)




The collection includes both original tweets and retweets. Since retweets mirrors the original tweet by someone else other than the author, we don't need to run sentimental analysis on the retweet as it would have been run on the original tweet, therefore we can split original tweets and retweets into two dataframes to avoid wasting resources.


- Original tweets: `dfOriginals`
- Retweets: `dfRetweets` cons


In [10]:
dfOriginals = df[df.retweet_id == 0]
dfRetweets = df[df.retweet_id != 0]

## Preview data after cleaning and manipulation

Original Tweet dataframe

In [11]:
dfOriginals.head(5)

Retweet dataframe

In [12]:
dfRetweets.head(5)

## Guessing language

In [13]:
#https://pypi.org/project/langdetect/
!pip install langdetect
from langdetect import detect
from langdetect import DetectorFactory
DetectorFactory.seed = 0

# import multiprocessing as mp
# p=mp.Pool(4)


     |████████████████████████████████| 983kB 5.9MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993195 sha256=93f518a9e73007c3fb8b5f9e1fc1a8f70691bbb7a6e1562c0749a140a5c71052
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [14]:
# p.map(detect,dfOriginals.sample(500).text)

In [15]:
dfOriginals = dfOriginals[~(dfOriginals.modified_text == "")]

In [16]:
dfOriginals[(dfOriginals.modified_text == "")]

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_screen_name,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified,modified_text


add markdown giving explanation

In [17]:
def is_en(txt):
    try:
        return detect(txt)=='en'
    except:
        return False

In [18]:
dfOriginals.sample(5).text.apply(is_en)

610219    True
185159    True
745618    True
40189     True
227659    True
Name: text, dtype: bool

In [ ]:
#https://stackoverflow.com/questions/60930935/exclude-non-english-rows-in-pandas

dfOriginals['guessed_language'] = dfOriginals.modified_text.apply(is_en)

# split to cores
# dfOriginals['guessed_language'] = p.map(is_en,dfOriginals.modified_text)
#dfOriginals.sample(1000).text.apply(detect)

- give extra information the analysis process
- what action did to the data

In [ ]:
sum(dfOriginals.guessed_language)

In [ ]:
len(dfOriginals)

# Being Sentiment Analysis

Import required libraries

In [ ]:
import nltk
from nltk.sentiment.util import *
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk import tokenize
nltk.download('punkt')
nltk.download('vader_lexicon')


In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
# for sentence in sentences:
#      print(sentence)
#      ss = sid.polarity_scores(sentence)
#      for k in sorted(ss):
#          print('{0}: {1}, '.format(k, ss[k]), end='')
#      print()

## Calculating Setimental Score



In [ ]:
dfOriginals['sentimentscore'] = dfOriginals.text.apply(sid.polarity_scores)

Quick preview of the sentiment score and its text.

In [ ]:
dfOriginals.sample(30)[['modified_text','sentimentscore']]

In [ ]:
# dfOriginals.sentimentscore.head(5)
dfOriginalSScore = pd.json_normalize(dfOriginals.sentimentscore)
dfOriginalSScore['original_index'] = dfOriginals.index
dfOriginalSScore = dfOriginalSScore.set_index('original_index')

In [ ]:
# dfOriginals.join(dfOriginalSScore)
dfOriginalSScore.head(5)

- need some explanation on the "compound" attribute
- have an answer what the compound score stands for.

In [ ]:
# pd.merge(dfOriginals, dfOriginalSScore, left_index=True, right_index='original_index')
dfOriginals.merge(dfOriginalSScore, left_index=True, right_index=True)

- impact factor of a tweet 